# Convolutional Neural Network on Fashion MNIST
---
Don't forget to use **https://pytorch.org/docs/stable/**

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

---

## Intro to convolutional filters

In [ ]:
from image_processing_workshop.utils import get_image_from_url
from image_processing_workshop.visual import plot_image
import numpy as np
import matplotlib.pylab as plt

### Download your favouirite image

In [ ]:
url = 'https://media.wired.com/photos/5bbf72c46278de2d2123485b/master/w_582,c_limit/soyuz-1051882240.jpg'
img = get_image_from_url(url, to_grayscale=True)
img = img / 255.
plot_image(img)

### Explore prepared filters

In [ ]:
initial_filter = np.array([[-1, -1, 1, 1], 
                           [-1, -1, 1, 1], 
                           [-1, -1, 1, 1], 
                           [-1, -1, 1, 1]])
filter_1 = initial_filter
filter_2 = -filter_1
filter_3 = filter_1.T
filter_4 = -filter_3
filters = np.array([filter_1, filter_2, filter_3, filter_4])

In [ ]:
fig = plt.figure(figsize=(10, 5))
for i in range(4):
    ax = fig.add_subplot(1, 4, i+1, xticks=[], yticks=[])
    ax.imshow(filters[i], cmap='gray')
    ax.set_title('Filter %s' % str(i+1))
    width, height = filters[i].shape
    
    # Add -1 1 annotations to image.
    for x in range(width):
        for y in range(height):
            ax.annotate(str(filters[i][x][y]), xy=(y,x),
                        horizontalalignment='center',
                        verticalalignment='center',
                        color='white' if filters[i][x][y]<0 else 'black')

### Build small network initialised with those filters
In the examples, we will use `torch.nn.conv2d` https://pytorch.org/docs/stable/nn.html#conv2d

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
# In PyTorch, we have channels on 1st. So here we have 4 filters, each has 1 channel, all are shape 4x4.
filters_torch = torch.from_numpy(filters).unsqueeze(1).type(torch.DoubleTensor)
filters_torch.shape

In [ ]:
img_torch = torch.from_numpy(img).unsqueeze(0).unsqueeze(1)
img_torch.shape

Convoluton filters efectively change height and width of input image that

$H_{out} = \lfloor \frac{H_{in}+2×padding[0]−dilation[0]×(kernel\_size[0]−1)−1}{stride[0]} +1 \rfloor$   
$W_{out} = \lfloor \frac{W_{in}+2×padding[1]−dilation[1]×(kernel\_size[1]−1)−1}{stride[1]} +1 \rfloor$



In [ ]:
class ConvNeuralNetSimple(nn.Module):    
    def __init__(self, filters_torch):
        super(ConvNeuralNetSimple, self).__init__()
        
        height, width = filters_torch.shape[2:]
        self.conv_layer = nn.Conv2d(in_channels=1, out_channels=4, 
                                    kernel_size=(height, width), bias=False)
        self.conv_layer.weight.data = filters_torch

    def forward(self, images):
        return self.conv_layer(images)
    
conv_neural_net_simple = ConvNeuralNetSimple(filters_torch)
print(conv_neural_net_simple)
print(list(conv_neural_net_simple.parameters()))

In [ ]:
img_torch.shape

In [ ]:
feature_maps = conv_neural_net_simple(img_torch)
feature_maps.shape

### Visualization of conv layer feature maps

In [ ]:
def vizualize_feature_maps(feature_maps, n_maps= 4):
    fig = plt.figure(figsize=(20, 20))
    
    for i in range(n_maps):
        ax = fig.add_subplot(1, n_maps, i+1, xticks=[], yticks=[])
        # grab layer outputs
        ax.imshow(np.squeeze(feature_maps[0,i].data.numpy()), cmap='gray')
        ax.set_title('Output %s' % str(i+1))

In [ ]:
# Source img.
plt.imshow(img, cmap='gray')

# Convolution filters.
fig = plt.figure(figsize=(12, 6))
fig.subplots_adjust(left=0, right=1.5, bottom=0.8, top=1, hspace=0.05, wspace=0.05)
for i in range(4):
    ax = fig.add_subplot(1, 4, i+1, xticks=[], yticks=[])
    ax.imshow(filters[i], cmap='gray')
    ax.set_title('Filter %s' % str(i+1))

# Feature maps.    
vizualize_feature_maps(feature_maps)

### Sensitivity of image on convolution filters

In [ ]:
from ipywidgets import interactive
import ipywidgets as ipw

In [ ]:
feature_map = feature_maps[0][0].detach().numpy()
feature_map.shape

In [ ]:
plot_image(filter_1, figsize=(5,5))

In [ ]:
feature_map_max = feature_map.max()
def plot_sensitivity(tolerance):
    feature_map_filtered = (feature_map >= (feature_map_max - tolerance)).astype(int)
    fig = plt.figure(figsize=(10, 10))
    im = plt.imshow(feature_map_filtered, cmap='gray')
    plt.colorbar(im, orientation='horizontal')
    plt.gca().axes.set_axis_off()
    plt.show()
    
interactive(plot_sensitivity, tolerance=ipw.FloatSlider(0.5, min=0, max=feature_map_max - 0.1, step=0.01))

---

## Prepare Fashion MNIST dataset

In [ ]:
from torchvision.datasets import FashionMNIST
from torchvision.transforms import ToTensor, Compose
from torch.utils.data import DataLoader

In [ ]:
transformations = Compose([ToTensor()])

train_dataset = FashionMNIST('./dataset_fashion_mnist/', download=True, train=True, 
                             transform=transformations, target_transform=None)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
valid_dataset = FashionMNIST('./dataset_fashion_mnist/', download=True, train=False, 
                             transform=transformations, target_transform=None)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=64, shuffle=False)

In [ ]:
valid_dataset[0][0].shape

---

## Define convolutional neural network

### Architecture

In [ ]:
from torch.nn import Module, Sequential
from torch.nn import ReLU, Tanh, Dropout, Softmax, Linear, Conv2d, MaxPool2d, BatchNorm2d
from torch.nn import MSELoss, CrossEntropyLoss, NLLLoss
from torch.optim import Adam, SGD
from torch.nn.init import xavier_uniform_, normal_

In [ ]:
class ConvNeuralNet(nn.Module):
    def __init__(self):
        super(ConvNeuralNet, self).__init__()
        # Variables for logging of layers shapes.
        self.use_softmax = False
        self.shape_conv1 = None

        # 1st segment of conv with batch norm and pooling.
        self.conv1 = nn.Sequential(
            Conv2d(1, 32, (3, 3), stride=(1, 1), padding=(1, 1)),
            BatchNorm2d(32),
            ReLU(),
            MaxPool2d((2, 2), stride=(2, 2)))

        #############################################################################
        # TODO: Add another convolution blocks (64 filters) and adjsut Linear part. #
        #############################################################################
        
        # Linear output.
        self.linear = Linear(14*14*32, 10)

    def forward(self, images):
        x = self.conv1(images)
        self.shape_conv1 = x.shape
        
        
        x = x.view(x.size(0), -1)
        x = self.linear(x)
        x = torch.log_softmax(x, dim=1)
        if self.use_softmax:
            return torch.exp(x)
        else:
            return x

conv_neural_net = ConvNeuralNet()
conv_neural_net

In [ ]:
valid_dataset[0][0]

In [ ]:
info = conv_neural_net.eval()

In [ ]:
conv_neural_net(valid_dataset[0][0].unsqueeze(0))

In [ ]:
conv_neural_net.use_softmax = True
conv_neural_net(valid_dataset[0][0].unsqueeze(0))

In [ ]:
conv_neural_net.use_softmax = False
info = conv_neural_net.train()

In [ ]:
conv_neural_net.shape_conv1

### Optimizers and loss function
More on loss functions can be found here: https://pytorch.org/docs/stable/nn.html#loss-functions  
More on optimizers can be found here: https://pytorch.org/docs/stable/optim.html

In [ ]:
loss_fce = NLLLoss()
loss_fce

In [ ]:
optimizer = Adam(conv_neural_net.parameters())
optimizer

### Training of neural net

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def get_valid_acc_and_loss(model, loss_fce, valid_loader):
    accuracy = 0
    loss = 0
    was_training = model.training
    
    model.eval()
    for images, labels in valid_loader:
        predictions = model(images)
        accuracy += (predictions.argmax(dim=1) == labels).type(torch.FloatTensor).mean().item() 
        loss += loss_fce(predictions, labels).item()
    model.train(mode=was_training)
    return accuracy / len(valid_loader) * 100, loss / len(valid_loader)

In [ ]:
get_valid_acc_and_loss(conv_neural_net, loss_fce, valid_loader)

In [ ]:
from collections import deque

# Initial params setup.
epochs = 2
report_period = 100
batch_iteration = 0

# Storing of some data.
train_leak_loss = deque(maxlen=report_period)
train_loss_history = []
valid_loss_history = []
valid_acc_history = []

In [ ]:
for epoch in range(epochs):
    # Setup net to train mode and go through one epoch.
    conv_neural_net.train()
    for images, labels in train_loader:
        batch_iteration += 1
        
        ##################
        # Training Phase #
        ##################
        optimizer.zero_grad()
        predictions = conv_neural_net.forward(images)
        loss = loss_fce(predictions, labels)
        loss.backward()
        optimizer.step()
        
        ####################
        # Validation Phase #
        ####################
        train_leak_loss.append(loss.item())
        if batch_iteration % report_period == 0:
            conv_neural_net.eval()
            
            # We don't want to collect info for gradients from here.
            with torch.no_grad():
                valid_accuracy, valid_loss = get_valid_acc_and_loss(conv_neural_net, loss_fce, valid_loader)
                
            print(f'Epoch: {epoch+1}/{epochs}.. ',
                  f"Train Loss: {round(np.mean(train_leak_loss), 2)}.. ",
                  f"Valid Loss: {round(valid_loss, 2)}.. ",
                  f"Valid Acc: {round(valid_accuracy, 2)}%")
            
            train_loss_history.append(np.mean(train_leak_loss))
            valid_loss_history.append(valid_loss)
            valid_acc_history.append(valid_accuracy)
                   
            conv_neural_net.train()

In [ ]:
fig = plt.figure(figsize=(10, 10))
ax = plt.gca()
ax.set_xlabel('Iteration')
ax.set_ylabel('Cross Entropy')
plt.plot(train_loss_history, label='Train loss')
plt.plot(valid_loss_history, label='Valid loss')
plt.legend(frameon=False)

In [ ]:
fig = plt.figure(figsize=(10, 10))
plt.plot(valid_acc_history, label='Valid acc')
ax = plt.gca()
ax.set_xlabel('Iteration')
ax.set_ylabel('Acc(%)')
plt.legend(frameon=False)

---

## Results evaluation

In [ ]:
conv_neural_net.eval()
conv_neural_net.use_softmax = True

### View single images and predictions

In [ ]:
from image_processing_workshop.visual import plot_classify, plot_image

In [ ]:
plot_classify(valid_dataset[1][0], conv_neural_net)

### Load reuslts to pandas df

In [ ]:
from image_processing_workshop.eval import get_results_df
from image_processing_workshop.visual import plot_df_examples

In [ ]:
df = get_results_df(conv_neural_net, valid_loader)
df.head(10)

In [ ]:
fig = plt.figure(figsize=(10, 10))
ax = plt.gca()
ax.set_xlabel('Prediction Score')
df[df.label_class_name=='Dress'].label_class_score.hist(ax=ax)

In [ ]:
plot_df_examples(df.iloc[:25])

### Precision

In [ ]:
from image_processing_workshop.eval import get_precision

In [ ]:
get_precision(df, 'Dress')

### Recall

In [ ]:
from image_processing_workshop.eval import get_recall

In [ ]:
get_recall(df, 'Dress')

### Overall Recall and Precision

In [ ]:
from image_processing_workshop.eval import get_rec_prec

In [ ]:
get_rec_prec(df)

### Accuracy

In [ ]:
from image_processing_workshop.eval import get_accuracy

In [ ]:
get_accuracy(df)

### False Positives


In [ ]:
from image_processing_workshop.eval import get_false_positives

In [ ]:
fp = get_false_positives(df, label_class_name='Shirt')

In [ ]:
plot_df_examples(fp)

In [ ]:
fp = get_false_positives(df, label_class_name='Shirt', predicted_class_name='Pullover')

In [ ]:
plot_df_examples(fp)

### Confusion Matrix

In [ ]:
from image_processing_workshop.visual import plot_coocurance_matrix

In [ ]:
plot_coocurance_matrix(df, use_log=False)